In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import itertools
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import coo_matrix

In [61]:
train = pd.read_csv("tourist_ml.csv")
train = train.dropna()
train.info
train

,Unnamed: 0,user_id,org_id,rating,ts,aspects,user_city,org_city
21,1865,1867414052571732177,1721607321278834453,4.0,286,"[4, 5]",spb,msk
30,3134,888845741213298786,13351482607452884539,5.0,945,[247],msk,spb
38,3177,18314704244139743583,13351482607452884539,4.0,623,[11],msk,spb
40,3179,1763402246693332519,13351482607452884539,5.0,201,[4],msk,spb
42,3181,7546272634645900434,13351482607452884539,5.0,700,[3],msk,spb
...,...,...,...,...,...,...,...,...
128888,3635996,4851604304432138986,7555496667092683636,5.0,429,"[22, 380]",msk,spb
128900,3636410,5966790683169678206,17259485834587658828,5.0,379,[6],msk,spb
128918,3637219,10019228002166718049,17884699404016676562,5.0,509,[6],msk,spb
128998,3640218,12418551340063486343,4528833785430195578,5.0,956,"[2, 4, 6, 19]",msk,spb


In [124]:
#Топ популярных мест в мск и спб
msk_orgs = train[(train['rating'] >= 4) & (train['org_city'] == 'msk')]['org_id']
msk_orgs = msk_orgs.value_counts().index[:19].to_list()

spb_orgs = train[(train['rating'] >= 4) & (train['org_city'] == 'spb')]['org_id']
spb_orgs = spb_orgs.value_counts().index[:19].to_list()

#spb_orgs = train[(train['user_id'] == 888845741213298786)]

print(type(spb_orgs))
print(len(spb_orgs))
spb_orgs


<class 'list'>
19


[2070377783033138991,
 12784724396885152442,
 18082340557576449859,
 4720145468635952379,
 4787182885298452451,
 707903906558057894,
 13534187238651328288,
 13351482607452884539,
 6102311996554982436,
 12909766905446674254,
 5756313314324569869,
 10148880420071080739,
 11012814543754137896,
 8790571376065654670,
 11010172576159263055,
 1902004535823209432,
 11078507181658784585,
 3136373007305862292,
 12982949732074223142]

In [ ]:
def taska(user_id):
    print(user_id)
    if len(train[train['user_id'] == user_id]) > 0:
        if len(train[(train['user_city'] == 'spb')]) > 0:
            spb_own = train[(train['rating'] >= 4) & (train['org_city'] == 'msk') & (train['user_id'] == user_id)]['org_id']
            #print("spb, ",spb_orgs[:9])
            try:
                if len(spb_own)>1:
                    all_orgs = spb_orgs[:9]+ spb_own.tolist
                    print("all",all_orgs[:19])
                    return all_orgs[:19]
                
                return spb_orgs[:9]
            except Exception as e:
                
                #print(spb_orgs[:9])
                #print("  ERROR  ")
                print(e, e.args)
        elif len(train[(train['user_city'] == 'msk')]) > 0:
            msk_own = train[(train['rating'] >= 4) & (train['org_city'] == 'spb') & (train['user_id'] == user_id)]['org_id']
            #print("msk",msk_orgs[:19])
            try:
                if len(msk_own)>1:
                    all_orgs = msk_orgs[:19] + msk_own.tolist()
                    return all_orgs[:19]
            except Exception as e:
                print("  ERROR  ")
                print(e,e.args)
            #print("msk",msk_orgs[:19])
            return msk_orgs[:19]

    else:
        return msk_orgs[:19]
        return spb_orgs[:19]
        print("none")

        

In [134]:

taska(888845741213298786)

888845741213298786
spb,  [2070377783033138991, 12784724396885152442, 18082340557576449859, 4720145468635952379, 4787182885298452451, 707903906558057894, 13534187238651328288, 13351482607452884539, 6102311996554982436]


[2070377783033138991,
 12784724396885152442,
 18082340557576449859,
 4720145468635952379,
 4787182885298452451,
 707903906558057894,
 13534187238651328288,
 13351482607452884539,
 6102311996554982436]

In [154]:
ycup=[]

In [203]:
test = pd.read_csv("test_users.csv")
ycup = pd.DataFrame({'user_id': [],
                   'target': []
                   })

for user in test["user_id"]:
    task = taska(user)[:19]
    print(len(task))
    task = str(task).replace(",", " ").replace("[","").replace("]","")
    #print(task)
    ycup.loc[len(ycup.index)] = [str(user),task]
    print(len(task.split(" ")))
    taska(user)[:19]
    print(" +++++")
    print(task.split(" "))
    



3545210947248911048
19
37
3545210947248911048
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
15271987121288045390
19
37
15271987121288045390
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '120817019123860

37
9720535111438299209
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
11384303333353054449
19
37
11384303333353054449
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398

 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
7717952486807961559
19
37
7717952486807961559
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '13162316

37
8285641187477826085
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
636253786372585698
19
37
636253786372585698
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '91747643987367

19
37
2787193163338909444
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
9097321931274499032
19
37
9097321931274499032
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '917476439

...

 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
15469618234616769527
19
37
15469618234616769527
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '131623

 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
10755501167022790449
19
37
10755501167022790449
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '131623

37
948859885541149092
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
11741462178723600937
19
37
11741462178723600937
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '91747643987

37
9532755952722299096
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398736746307', '', '1316231655856055291', '', '1290456531260424598', '', '16727131031804531776', '', '5964742790934613222']
15359770849909594079
19
37
15359770849909594079
 +++++
['11229813210509740706', '', '15684663803879321952', '', '15191107094356976843', '', '11063400822680612569', '', '1625971115460696067', '', '4597033878882788970', '', '379209346827481342', '', '13573322486152844808', '', '1852569111777211913', '', '10591254745142129893', '', '16989189961318045230', '', '5346498272260997407', '', '9793490370400998070', '', '12081701912386059015', '', '9174764398

KeyboardInterrupt: 

In [199]:
i=0
for row in ycup["target"]:
    if len(row.split(" ")) >20:
        i+=1
print(i)
ycup

15532


,user_id,target
0,3545210947248911048,11229813210509740706 15684663803879321952 15...
1,15271987121288045390,11229813210509740706 15684663803879321952 15...
2,15016858616184265932,11229813210509740706 15684663803879321952 15...
3,12457244142928722989,11229813210509740706 15684663803879321952 15...
4,13339684649926251468,11229813210509740706 15684663803879321952 15...
...,...,...
16962,1191875913294598364,11229813210509740706 15684663803879321952 15...
16963,3866507700167344338,11229813210509740706 15684663803879321952 15...
16964,11434952144484188987,11229813210509740706 15684663803879321952 15...
16965,7010426792722803474,11229813210509740706 15684663803879321952 15...


In [204]:
a=[1,2,3,4]
b=[5,6,7]
c = a+b
for x in c:
    print(x)

1
2
3
4
5
6
7


In [206]:
ycup.to_csv('answer_top_3.csv', index=False)